## Titanic Dataset 
This is my first kaggle competition and since this dataset has been studied and used as an example so many times, I will jump straight ahead to the analysis just to see how making a submission and subsequent ranking in Kaggle works.

In [8]:
list1 = [1,2,4]
list1[0] = 'A'
print(list1)



['A', 2, 4]


NameError: name 'tup' is not defined

In [98]:
import pandas as pd 
#import data 
train = pd.read_csv('/Users/elbagomeznavas/Desktop/GITHUB/Kaggle-Practice/Titanic/train.csv')
test = pd.read_csv('/Users/elbagomeznavas/Desktop/GITHUB/Kaggle-Practice/Titanic/test.csv')
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


It looks like we will have to clean our data a little to remove variables that are not very useful, like 'Cabin'and 'Name'among others. Our outcome variable is survival so we set our features and outcome accordingly. 


In [102]:
# Create X, y vectors 
y_train = train['Survived']
print(y_train.dtype)
X_train = train.drop(['Survived', 'Cabin', 'Name','Ticket', 'PassengerId'], axis=1)
X_test = test.drop(['Cabin', 'Name', 'Ticket', 'PassengerId'], axis=1)

# We replace the missing values with the mean 
x_a =  X_test['Fare'].mean()
X_test['Fare'] = X_test['Fare'].fillna(x_a)
#Convert the fare values to integers 
X_train['Fare'] =  X_train['Fare'].astype(int)
X_test['Fare'] =  X_test['Fare'].astype(int)

#Convert Sex and Embarked categorical variables to Binary
def convert(x):
    if x == 'male' or x == 'Q':
        return(1)
    else:
        return(0)
    
X_train['Sex'] = X_train['Sex'].apply(convert)
X_train['Embarked'] = X_train['Embarked'].apply(convert)

X_test['Sex'] = X_test['Sex'].apply(convert)
X_test['Embarked'] = X_test['Embarked'].apply(convert)

print(X_train.info())
print(X_test.info())

int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null int64
Embarked    891 non-null int64
dtypes: float64(1), int64(6)
memory usage: 48.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null int64
Embarked    418 non-null int64
dtypes: float64(1), int64(6)
memory usage: 22.9 KB
None


### Missing values 
Something that will take slightly more nuance is fixing the missing values present in the 'Age' variable. There are many different ways to deal with missing values for random forest. The simplest one is passing the mean or mode of the variable to any missing value. A more sophisticated approach is doing KNN. I will opt for the simpler approach since I just want to make my first submission. 

Here's a good overview for dealing with N/A's in RF: https://medium.com/airbnb-engineering/overcoming-missing-values-in-a-random-forest-classifier-7b1fc1fc03ba 

In [103]:
#Now, we want to deal with missing values
x = X_train['Age'].mean()
x_1 = X_test['Age'].mean()

# We replace the missing values with the mean 
X_train['Age'] = X_train['Age'].fillna(x)
X_test['Age'] = X_test['Age'].fillna(x_1)

In [104]:
#Fit random forest model 
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 100 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [105]:
#Write as dataset for submission 
pass_id = test['PassengerId']

data = {"Survived" : predictions, "PassengerId": pass_id}

submission = pd.DataFrame.from_dict(data)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [106]:
submission.to_csv('/Users/elbagomeznavas/Desktop/GITHUB/Kaggle-Practice/Titanic/submission.csv', index = False) 